# Mixtures of MNLs

In [928]:
import gurobipy as gp

In [929]:
import numpy as np
import numpy.random as npr

N = 3 #5 #25 50 75 100
M = 2 #1 5 10 20
v = np.zeros((N, M))
for i in range(N):
    for j in range(M):
        v[i, j] = round((npr.randint(0, 2) + npr.rand()*00.1), 4)
theta = list(np.random.dirichlet(np.ones(M), size=1)[0])
print(v, theta)

[[0.0938 1.0841]
 [0.0503 1.0829]
 [1.005  0.0832]] [0.918869589674379, 0.08113041032562104]


In [930]:
def profit_mmnl(p, S):
    '''
    This function computes the profit of a given assortment S
    INPUT:
    p: list of prices
    S: assortment
    OUTPUT:
    pi: profit
    '''
    global v
    global theta, M
    pi = 0
    for j in range(M):
        den = 1
        num = 0
        for i in S:
            num += p[i]*v[i][j]
            den += v[i][j]
        pi += theta[j]*num/den
    return pi

In [931]:
def IP_mmnl(p):
    '''
    This function solves the IP model for the MMNL
    INPUT:
    p: list of prices
    OUTPUT:
    profit: best profit
    bestS: best assortment
    '''
    global v, N
    global theta, M

    # create a new model
    model = gp.Model("IP_mmnl")

    # create decision variables and store them in the arrays z, x, y
    z = model.addVars(M, vtype=gp.GRB.CONTINUOUS, lb=0, name="z")
    x = model.addVars(N, M, vtype=gp.GRB.CONTINUOUS, lb=0, name="x")
    y = model.addVars(N, vtype=gp.GRB.BINARY, name="y")
    model.update()

    # set objective function
    objExpr = gp.LinExpr()
    for j in range(M):
        objExpr += theta[j] * z[j]
    model.setObjective(objExpr, gp.GRB.MAXIMIZE)

    B = 10*max(p)
    # add constraints
    for j in range(M):
        myConstr = gp.LinExpr()
        for i in range(N):
            myConstr += v[i][j] * x[i, j]
            model.addConstr(lhs = x[i, j], sense = gp.GRB.LESS_EQUAL, rhs = y[i]*B, name = "xa_" + str(i) + '_' + str(j))
            model.addConstr(lhs = x[i, j], sense = gp.GRB.GREATER_EQUAL, rhs = (p[i] - z[j]) - (1-y[i])*B, name = "xb_" + str(i) + '_' + str(j))
            model.addConstr(lhs = x[i, j], sense = gp.GRB.LESS_EQUAL, rhs = (p[i] - z[j]) + (1-y[i])*B, name = "xc_" + str(i) + '_' + str(j))
        model.addConstr(lhs = z[j], sense = gp.GRB.EQUAL, rhs = myConstr, name = "z_" + str(j))
    model.update()

    # write model to file
    model.write("IP_mmnl.lp")

    # solve model
    model.Params.OutputFlag = 0
    model.optimize()
    
    bestS = []
    profit = float(model.objVal)
    for i in range(N):
        if y[i].x == 1:
            bestS.append(i)
    return profit, bestS

In [932]:
def brute_force(p):
    global N
    # compute the profit of each assortment
    best_profit = 0
    best_S = []
    for i in range(2**N):
        binary = np.binary_repr(i, width=N)
        assortment = [int(x) for x in binary]
        S = []
        for j in range(N):
            if assortment[j] == 1:
                S.append(j)
        if best_profit < profit_mmnl(p, S):
            best_profit = profit_mmnl(p, S)
            best_S = S
        print(i, S, profit_mmnl(p, S))
    return best_profit, best_S

In [933]:
def nested_by_price_mmnl(p):
    global N
    profits = [0 for i in range(N)]
    S = []
    for i in range(N):
        S.append(i)
        profits[i] = profit_mmnl(p, S)
    best_S = [i for i in range(np.argmax(profits)+1)]
    return max(profits), best_S

In [934]:
def find_best_option_mmnl(p, S):
    global v, N
    current_best = profit_mmnl(p, S)
    bestS = S
    flag = 0
    N_S = [k for k in range(N) if k not in S]
    for k in N_S:
        newS = S + [k]
        if profit_mmnl(p, newS) >= current_best:
            flag = 1
            bestS = newS
            current_best = profit_mmnl(p, newS)
    return current_best, bestS, flag

def greedy_mmnl(p):
    global v, N
    max_profit = -1
    best_S = []
    S = []
    flag = 1
    while flag==1:
        max_profit, S, flag = find_best_option_mmnl(p, S)
    best_S = sorted(S)
    return max_profit, best_S

In [935]:
import matplotlib.pyplot as plt
import math
import statistics as st
import time

In [936]:
n_simulations = 1000
avg_price = 10
std_price = 5
#compute prices
psim = [0 for k in range(n_simulations)]
for k in range(n_simulations):
    # psim[k] = abs(npr.normal(avg_price, std_price, N))
    # psim[k] = npr.uniform(0, 2*avg_price, N)
    psim[k] = npr.exponential(avg_price, N)
    # round prices
    for i in range(N):
        psim[k][i] = round(psim[k][i])
    psim[k] = sorted(psim[k], reverse=True)

In [937]:
# problem solved using the MIP formulation
profits_IP = []
times_IP = []
bestS_IP = []
for k in range(n_simulations):
    start_time = time.time()
    profit, bestS = IP_mmnl(psim[k])
    end_time = time.time()
    profits_IP.append(profit)
    times_IP.append(end_time-start_time)
    bestS_IP.append(bestS)
avg_time_IP = st.mean(times_IP)

In [938]:
# problem solved using the nested formulation
profits_nested = []
times_nested = []
bestS_nested = []
for k in range(n_simulations):
    start_time = time.time()
    profit, bestS = nested_by_price_mmnl(psim[k])
    end_time = time.time()
    profits_nested.append(profit)
    times_nested.append(end_time-start_time)
    bestS_nested.append(bestS)
avg_time_nested = st.mean(times_nested)

In [939]:
# problem solved using the greedy formulation
profits_greedy = []
times_greedy = []
bestS_greedy = []
for k in range(n_simulations):
    start_time = time.time()
    profit, bestS = greedy_mmnl(psim[k])
    end_time = time.time()
    profits_greedy.append(profit)
    times_greedy.append(end_time-start_time)
    bestS_greedy.append(bestS)
avg_time_greedy = st.mean(times_greedy)

In [940]:
# counting how many instances are different
count_nested = 0
different_results_nested = []
count_greedy = 0
different_results_greedy = []
for k in range(n_simulations):
    if np.isclose(profits_greedy[k], profits_IP[k], rtol=1e-3) == False:
        count_greedy += 100/n_simulations
        different_results_greedy.append(100*profits_greedy[k]/profits_IP[k])
        print(k, bestS_greedy[k], profits_greedy[k], bestS_IP[k], profits_IP[k], bestS_nested[k], profits_nested[k])
    if np.isclose(profits_nested[k], profits_IP[k], rtol=1e-3) == False:
        count_nested += 100/n_simulations
        different_results_nested.append(100*profits_nested[k]/profits_IP[k])
        print(k, bestS_greedy[k], profits_greedy[k], bestS_IP[k], profits_IP[k], bestS_nested[k], profits_nested[k])

4 [0, 1, 2] 3.170295723989552 [1, 2] 2.703455635505771 [0, 1, 2] 3.170295723989552
4 [0, 1, 2] 3.170295723989552 [1, 2] 2.703455635505771 [0, 1, 2] 3.170295723989552
6 [0, 1, 2] 4.89456642564153 [0, 1] 4.0785059099062355 [0, 1, 2] 4.89456642564153
6 [0, 1, 2] 4.89456642564153 [0, 1] 4.0785059099062355 [0, 1, 2] 4.89456642564153
7 [0, 1, 2] 2.123876690989697 [2] 1.4004363038500252 [0, 1, 2] 2.123876690989697
7 [0, 1, 2] 2.123876690989697 [2] 1.4004363038500252 [0, 1, 2] 2.123876690989697
8 [0, 1] 2.1266710080886746 [0] 1.9360128965151464 [0, 1] 2.1266710080886746
8 [0, 1] 2.1266710080886746 [0] 1.9360128965151464 [0, 1] 2.1266710080886746
11 [0, 1, 2] 2.7210222876929815 [0, 1] 2.124875498972451 [0, 1, 2] 2.721022287692981
11 [0, 1, 2] 2.7210222876929815 [0, 1] 2.124875498972451 [0, 1, 2] 2.721022287692981
12 [0, 1, 2] 0.6146442382023066 [1, 2] 0.5154721791197094 [0, 1, 2] 0.6146442382023066
12 [0, 1, 2] 0.6146442382023066 [1, 2] 0.5154721791197094 [0, 1, 2] 0.6146442382023066
13 [0, 2] 

In [941]:
if different_results_greedy != []:
  print('differences greedy: ' + str(count_greedy))
  print('average ratio: ' + str(st.mean(different_results_greedy)))
  print('minimum ratio: ' + str(min(different_results_greedy)))
  print('average time: ' + str(avg_time_greedy) + '\n')
else:
  print('differences greedy: ' + str(count_greedy))
  print('average ratio: 100')
  print('minimum ratio: 100')
  print('average time: ' + str(avg_time_greedy) + '\n')

if different_results_nested != []:
  print('differences nested: ' + str(count_nested))
  print('average ratio: ' + str(st.mean(different_results_nested)))
  print('minimum ratio: ' + str(min(different_results_nested)))
  print('average time: ' + str(avg_time_nested))
else:
  print('differences nested: ' + str(count_nested))
  print('average ratio: 100')
  print('minimum ratio: 100')
  print('average time: ' + str(avg_time_nested))

differences greedy: 67.60000000000043
average ratio: 125.31078540790317
minimum ratio: 100.54098011907834
average time: 8.508181571960449e-05

differences nested: 66.5000000000005
average ratio: 125.09739268438342
minimum ratio: 100.54098011907834
average time: 3.26697826385498e-05


In [942]:
# brute_force(psim[0])


In [945]:
model = gp.Model("IP_mmnl2")
p = psim[999]
# create decision variables and store them in the arrays z, x, y
z = model.addVars(M, vtype=gp.GRB.CONTINUOUS, lb=0, name="z")
x = model.addVars(N, M, vtype=gp.GRB.CONTINUOUS, lb=0, name="x")
y = model.addVars(N, vtype=gp.GRB.BINARY, name="y")
model.update()

# set objective function
objExpr = gp.LinExpr()
for j in range(M):
    objExpr += theta[j] * z[j]
model.setObjective(objExpr, gp.GRB.MAXIMIZE)

for i in range(N):
    model.addConstr(lhs = y[i], sense = gp.GRB.EQUAL, rhs = 1, name = "y_" + str(i))
B = max(p)+1
for j in range(M):
    myConstr = gp.LinExpr()
    for i in range(N):
        myConstr += v[i][j] * x[i, j]
        model.addConstr(lhs = x[i, j], sense = gp.GRB.LESS_EQUAL, rhs = y[i]*B, name = "xa_" + str(i) + '_' + str(j))
        model.addConstr(lhs = x[i, j], sense = gp.GRB.GREATER_EQUAL, rhs = (p[i] - z[j]) - (1-y[i])*B, name = "xb_" + str(i) + '_' + str(j))
        model.addConstr(lhs = x[i, j], sense = gp.GRB.LESS_EQUAL, rhs = (p[i] - z[j]) + (1-y[i])*B, name = "xc_" + str(i) + '_' + str(j))
    model.addConstr(lhs = z[j], sense = gp.GRB.EQUAL, rhs = myConstr, name = "z_" + str(j))
model.update()

# solve model
model.Params.OutputFlag = 0
model.Params.FeasibilityTol = 1e-2
model.Params.OptimalityTol = 1e-2
model.optimize()

# write model to file
model.write('IP_mmnl2.lp')

# print solution
if model.status == gp.GRB.Status.OPTIMAL:
    print('Optimal objective: %g' % model.objVal)
    for value in model.getVars():
        print('%s %g' % (value.varName, value.x))